In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import numpy as np
import statsmodels.api as sm
from pylab import rcParams

In [69]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('D:\Capstone\Hourly_Demand_Clean_FINAL.csv')

In [3]:
df

,date,hour,participant,resource,mw,price,year,month,day,weekday,hour_int,date_hour
0,2019-01-01 00:00:00,0 days 00:00:00.000000000,WESM,1BALNEO_L01,50.4,NaN,2019,1,1,1,0,2019-01-01 00:00:00
1,2019-01-01 00:00:00,0 days 00:00:00.000000000,WESM,1BINGA_T1L1,0.0,NaN,2019,1,1,1,0,2019-01-01 00:00:00
2,2019-01-01 00:00:00,0 days 00:00:00.000000000,WESM,1BOTOL_T1L2,13.8,NaN,2019,1,1,1,0,2019-01-01 00:00:00
3,2019-01-01 00:00:00,0 days 00:00:00.000000000,WESMTMP,1CBNTU_T1L1,33.3,NaN,2019,1,1,1,0,2019-01-01 00:00:00
4,2019-01-01 00:00:00,0 days 00:00:00.000000000,WESM,1CBN_CELCOR,24.3,NaN,2019,1,1,1,0,2019-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2445171,2020-03-28 00:00:00,0 days 23:00:00.000000000,WESM,3MAKBA_T3L1,1.0,2307.96,2020,3,28,5,23,2020-03-28 23:00:00
2445172,2020-03-28 00:00:00,0 days 23:00:00.000000000,WESM,3NAG_CSRC2B,8.0,2182.42,2020,3,28,5,23,2020-03-28 23:00:00
2445173,2020-03-28 00:00:00,0 days 23:00:00.000000000,WESM,3NAG_CSRCO2,27.5,2182.42,2020,3,28,5,23,2020-03-28 23:00:00
2445174,2020-03-28 00:00:00,0 days 23:00:00.000000000,WESM,3ORMAT_SS,0.0,2307.96,2020,3,28,5,23,2020-03-28 23:00:00


In [4]:
df.columns

Index(['date', 'hour', 'participant', 'resource', 'mw', 'price', 'year',
       'month', 'day', 'weekday', 'hour_int', 'date_hour'],
      dtype='object')

In [5]:
df['date'] =  pd.to_datetime(df['date'], format='%Y/%m/%d',infer_datetime_format=True)

In [6]:
df

,date,hour,participant,resource,mw,price,year,month,day,weekday,hour_int,date_hour
0,2019-01-01,0 days 00:00:00.000000000,WESM,1BALNEO_L01,50.4,NaN,2019,1,1,1,0,2019-01-01 00:00:00
1,2019-01-01,0 days 00:00:00.000000000,WESM,1BINGA_T1L1,0.0,NaN,2019,1,1,1,0,2019-01-01 00:00:00
2,2019-01-01,0 days 00:00:00.000000000,WESM,1BOTOL_T1L2,13.8,NaN,2019,1,1,1,0,2019-01-01 00:00:00
3,2019-01-01,0 days 00:00:00.000000000,WESMTMP,1CBNTU_T1L1,33.3,NaN,2019,1,1,1,0,2019-01-01 00:00:00
4,2019-01-01,0 days 00:00:00.000000000,WESM,1CBN_CELCOR,24.3,NaN,2019,1,1,1,0,2019-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2445171,2020-03-28,0 days 23:00:00.000000000,WESM,3MAKBA_T3L1,1.0,2307.96,2020,3,28,5,23,2020-03-28 23:00:00
2445172,2020-03-28,0 days 23:00:00.000000000,WESM,3NAG_CSRC2B,8.0,2182.42,2020,3,28,5,23,2020-03-28 23:00:00
2445173,2020-03-28,0 days 23:00:00.000000000,WESM,3NAG_CSRCO2,27.5,2182.42,2020,3,28,5,23,2020-03-28 23:00:00
2445174,2020-03-28,0 days 23:00:00.000000000,WESM,3ORMAT_SS,0.0,2307.96,2020,3,28,5,23,2020-03-28 23:00:00


In [7]:
df.dtypes

date           datetime64[ns]
hour                   object
participant            object
resource               object
mw                    float64
price                 float64
year                    int64
month                   int64
day                     int64
weekday                 int64
hour_int                int64
date_hour              object
dtype: object

In [8]:
df = df.drop(['hour','price','year','month','day','weekday','hour_int','date_hour'], axis=1)

In [9]:
df

,date,participant,resource,mw
0,2019-01-01,WESM,1BALNEO_L01,50.4
1,2019-01-01,WESM,1BINGA_T1L1,0.0
2,2019-01-01,WESM,1BOTOL_T1L2,13.8
3,2019-01-01,WESMTMP,1CBNTU_T1L1,33.3
4,2019-01-01,WESM,1CBN_CELCOR,24.3
...,...,...,...,...
2445171,2020-03-28,WESM,3MAKBA_T3L1,1.0
2445172,2020-03-28,WESM,3NAG_CSRC2B,8.0
2445173,2020-03-28,WESM,3NAG_CSRCO2,27.5
2445174,2020-03-28,WESM,3ORMAT_SS,0.0


In [10]:
df['totalday'] = df.groupby(['date','resource'])['mw'].transform('sum')

In [11]:
df

,date,participant,resource,mw,totalday
0,2019-01-01,WESM,1BALNEO_L01,50.4,1089.8
1,2019-01-01,WESM,1BINGA_T1L1,0.0,0.0
2,2019-01-01,WESM,1BOTOL_T1L2,13.8,295.8
3,2019-01-01,WESMTMP,1CBNTU_T1L1,33.3,698.3
4,2019-01-01,WESM,1CBN_CELCOR,24.3,587.6
...,...,...,...,...,...
2445171,2020-03-28,WESM,3MAKBA_T3L1,1.0,26.6
2445172,2020-03-28,WESM,3NAG_CSRC2B,8.0,194.4
2445173,2020-03-28,WESM,3NAG_CSRCO2,27.5,677.3
2445174,2020-03-28,WESM,3ORMAT_SS,0.0,0.0


In [12]:
df = df.sort_values(by='date')

In [13]:
df

,date,participant,resource,mw,totalday
0,2019-01-01,WESM,1BALNEO_L01,50.4,1089.8
342411,2019-01-01,WESM,1ANGAT_SS,3.7,27.8
342412,2019-01-01,WESM,1BNTAY_T1L1,13.9,291.1
342413,2019-01-01,WESMTMP,1BNTAY_T1L3,0.0,0.0
342414,2019-01-01,WESM,1BYOMB_T1L3,27.5,586.7
...,...,...,...,...,...
1729403,2020-07-15,WESM,1HERMO_T1L2,0.0,0.0
1729402,2020-07-15,WESM,1HERMO_T1L1,33.7,852.3
1729401,2020-07-15,WESM,1GAMU_T1L2,0.0,8.4
1729407,2020-07-15,WESM,1NSNMN_T1L1,20.7,500.8


In [14]:
#For checking
DF_1BALNEO_L01 = df[df['resource'] == '1BALNEO_L01']
DF_1BALNEO_L01

,date,participant,resource,mw,totalday
0,2019-01-01,WESM,1BALNEO_L01,50.4,1089.8
342473,2019-01-01,WESM,1BALNEO_L01,53.0,1089.8
342453,2019-01-01,WESM,1BALNEO_L01,54.8,1089.8
342309,2019-01-01,WESM,1BALNEO_L01,48.2,1089.8
98754,2019-01-01,WESM,1BALNEO_L01,49.2,1089.8
...,...,...,...,...,...
1665162,2020-07-15,WESM,1BALNEO_L01,20.4,610.8
1665198,2020-07-15,WESM,1BALNEO_L01,17.9,610.8
1729481,2020-07-15,WESM,1BALNEO_L01,26.6,610.8
1729379,2020-07-15,WESM,1BALNEO_L01,25.7,610.8


In [15]:
#For checking
DF_1BALNEO_L01 = df[df['resource'] == '1BALNEO_L01']
DF_1BALNEO_L01[0:25]

,date,participant,resource,mw,totalday
0,2019-01-01,WESM,1BALNEO_L01,50.4,1089.8
342473,2019-01-01,WESM,1BALNEO_L01,53.0,1089.8
342453,2019-01-01,WESM,1BALNEO_L01,54.8,1089.8
342309,2019-01-01,WESM,1BALNEO_L01,48.2,1089.8
98754,2019-01-01,WESM,1BALNEO_L01,49.2,1089.8
342541,2019-01-01,WESM,1BALNEO_L01,41.5,1089.8
147603,2019-01-01,WESM,1BALNEO_L01,37.2,1089.8
147629,2019-01-01,WESM,1BALNEO_L01,39.5,1089.8
342175,2019-01-01,WESM,1BALNEO_L01,36.1,1089.8
342080,2019-01-01,WESM,1BALNEO_L01,45.4,1089.8


In [16]:
#Fore checking
DailyDF_1BALNEO_L01 = DF_1BALNEO_L01.groupby('date')['mw'].agg('sum')

In [17]:
#For checking
DailyDF_1BALNEO_L01
#Sums Matched

date
2019-01-01    1089.8
2019-01-02    1206.4
2019-01-03    1335.7
2019-01-04    1348.1
2019-01-05    1288.1
               ...  
2020-07-11     574.0
2020-07-12     534.7
2020-07-13     554.3
2020-07-14     574.1
2020-07-15     610.8
Name: mw, Length: 562, dtype: float64

In [18]:
df

,date,participant,resource,mw,totalday
0,2019-01-01,WESM,1BALNEO_L01,50.4,1089.8
342411,2019-01-01,WESM,1ANGAT_SS,3.7,27.8
342412,2019-01-01,WESM,1BNTAY_T1L1,13.9,291.1
342413,2019-01-01,WESMTMP,1BNTAY_T1L3,0.0,0.0
342414,2019-01-01,WESM,1BYOMB_T1L3,27.5,586.7
...,...,...,...,...,...
1729403,2020-07-15,WESM,1HERMO_T1L2,0.0,0.0
1729402,2020-07-15,WESM,1HERMO_T1L1,33.7,852.3
1729401,2020-07-15,WESM,1GAMU_T1L2,0.0,8.4
1729407,2020-07-15,WESM,1NSNMN_T1L1,20.7,500.8


In [19]:
df = df.drop(['mw'], axis=1)
df

,date,participant,resource,totalday
0,2019-01-01,WESM,1BALNEO_L01,1089.8
342411,2019-01-01,WESM,1ANGAT_SS,27.8
342412,2019-01-01,WESM,1BNTAY_T1L1,291.1
342413,2019-01-01,WESMTMP,1BNTAY_T1L3,0.0
342414,2019-01-01,WESM,1BYOMB_T1L3,586.7
...,...,...,...,...
1729403,2020-07-15,WESM,1HERMO_T1L2,0.0
1729402,2020-07-15,WESM,1HERMO_T1L1,852.3
1729401,2020-07-15,WESM,1GAMU_T1L2,8.4
1729407,2020-07-15,WESM,1NSNMN_T1L1,500.8


In [20]:
df = df.rename(columns={'totalday':'total mw'})

In [21]:
df

,date,participant,resource,total mw
0,2019-01-01,WESM,1BALNEO_L01,1089.8
342411,2019-01-01,WESM,1ANGAT_SS,27.8
342412,2019-01-01,WESM,1BNTAY_T1L1,291.1
342413,2019-01-01,WESMTMP,1BNTAY_T1L3,0.0
342414,2019-01-01,WESM,1BYOMB_T1L3,586.7
...,...,...,...,...
1729403,2020-07-15,WESM,1HERMO_T1L2,0.0
1729402,2020-07-15,WESM,1HERMO_T1L1,852.3
1729401,2020-07-15,WESM,1GAMU_T1L2,8.4
1729407,2020-07-15,WESM,1NSNMN_T1L1,500.8


In [22]:
df.dtypes

date           datetime64[ns]
participant            object
resource               object
total mw              float64
dtype: object

In [23]:
df = df.sort_values(by='date')

In [24]:
df

,date,participant,resource,total mw
0,2019-01-01,WESM,1BALNEO_L01,1089.8
196100,2019-01-01,WESM,1SUAL_T3L1,109.5
196099,2019-01-01,WESM,1SNEST_T3L2,139.2
196098,2019-01-01,WESM,1SNEST_T3L1,275.5
196097,2019-01-01,WESM,1MSNLO_BATC,0.0
...,...,...,...,...
1639740,2020-07-15,WESM,3BATAN_T1L2,274.7
1639741,2020-07-15,MERALCO,3BINAN_T1L1,11893.1
1639742,2020-07-15,ALECO,3DAR_ALECO,931.1
1639736,2020-07-15,WESMTMP,1MEX_PELCO1,726.4


In [25]:
df = df.drop_duplicates()

In [26]:
df[df['resource'] == '3MAKBA_T3L1']

,date,participant,resource,total mw
147902,2019-01-01,WESM,3MAKBA_T3L1,21.9
994,2019-01-02,WESM,3MAKBA_T3L1,24.8
343172,2019-01-03,WESM,3MAKBA_T3L1,52.7
246043,2019-01-04,WESM,3MAKBA_T3L1,51.5
52048,2019-01-05,WESM,3MAKBA_T3L1,26.3
...,...,...,...,...
1682122,2020-07-11,WESM,3MAKBA_T3L1,23.1
1816824,2020-07-12,WESM,3MAKBA_T3L1,24.0
1873214,2020-07-13,WESM,3MAKBA_T3L1,36.5
1704118,2020-07-14,WESM,3MAKBA_T3L1,48.5


In [27]:
Daily_DF = df.copy()

In [28]:
Daily_DF

,date,participant,resource,total mw
0,2019-01-01,WESM,1BALNEO_L01,1089.8
196100,2019-01-01,WESM,1SUAL_T3L1,109.5
196099,2019-01-01,WESM,1SNEST_T3L2,139.2
196098,2019-01-01,WESM,1SNEST_T3L1,275.5
196097,2019-01-01,WESM,1MSNLO_BATC,0.0
...,...,...,...,...
1889990,2020-07-15,WESM,1MRVEL_T1L1,0.0
1889891,2020-07-15,WESM,3BAT_BTLEC2,763.7
1889872,2020-07-15,WESM,3CALIRY_SS,0.0
1889814,2020-07-15,WESM,1SUAL_T3L1,108.4


In [29]:
cluster_df = pd.read_csv('D:\Capstone\df_clustered.csv')

In [30]:
cluster_df

,cluster,reso_id,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,0,1AEC_L01,0.322785,0.202532,0.101266,0.034810,0.000000,0.018987,0.000000,0.164557,...,0.949367,0.981013,0.873418,0.905063,1.000000,0.962025,0.911392,0.791139,0.664557,0.487342
1,0,1BAUAN_T2L1,0.200000,0.150000,0.100000,0.050000,0.150000,0.200000,0.000000,0.100000,...,0.500000,0.450000,0.350000,0.550000,0.950000,1.000000,0.900000,0.700000,0.450000,0.300000
2,0,1BAUAN_T2L2,0.244681,0.170213,0.074468,0.026596,0.042553,0.085106,0.000000,0.202128,...,0.797872,0.723404,0.563830,0.627660,0.946809,1.000000,0.914894,0.723404,0.505319,0.351064
3,0,1BCNOT_T1L2,0.438596,0.350877,0.271930,0.210526,0.228070,0.192982,0.000000,0.052632,...,0.473684,0.385965,0.280702,0.491228,0.912281,1.000000,0.947368,0.771930,0.684211,0.543860
4,0,1BNTAY_T1L2,0.270270,0.189189,0.108108,0.027027,0.054054,0.108108,0.000000,0.108108,...,0.567568,0.459459,0.378378,0.486486,0.864865,1.000000,0.972973,0.756757,0.567568,0.405405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,5,1MEX_SKK,0.926108,0.949097,0.903120,0.901478,0.866995,0.880131,0.801314,0.866995,...,0.016420,0.492611,0.541872,0.558292,0.650246,0.840722,0.932677,0.853859,1.000000,0.899836
149,5,1NSNMN_T1L1,0.782609,0.782609,0.782609,0.913043,0.869565,1.000000,0.739130,0.608696,...,0.652174,0.521739,0.782609,0.608696,0.739130,0.826087,0.913043,0.869565,0.913043,0.565217
150,5,1SNJOS_T1L1,0.538462,0.769231,0.730769,0.846154,1.000000,0.769231,0.923077,0.769231,...,0.461538,0.461538,0.000000,0.461538,0.384615,0.692308,0.846154,0.769231,0.769231,0.538462
151,5,1SNJOS_T1L3,0.710526,0.763158,0.868421,0.907895,1.000000,0.921053,0.842105,0.697368,...,0.065789,0.026316,0.315789,0.289474,0.157895,0.250000,0.355263,0.473684,0.611842,0.710526


In [31]:
x = []
for i in range(0,24):
    x.append(str(i))

cluster_df = cluster_df.drop(columns = x)
cluster_df

,cluster,reso_id
0,0,1AEC_L01
1,0,1BAUAN_T2L1
2,0,1BAUAN_T2L2
3,0,1BCNOT_T1L2
4,0,1BNTAY_T1L2
...,...,...
148,5,1MEX_SKK
149,5,1NSNMN_T1L1
150,5,1SNJOS_T1L1
151,5,1SNJOS_T1L3


In [36]:
cluster_dic = pd.Series(cluster_df.cluster.values,index=cluster_df.reso_id).to_dict()

In [37]:
Daily_DF['cluster'] = Daily_DF['resource'].map(cluster_dic)

In [38]:
Daily_DF

,date,participant,resource,total mw,cluster
0,2019-01-01,WESM,1BALNEO_L01,1089.8,2.0
196100,2019-01-01,WESM,1SUAL_T3L1,109.5,4.0
196099,2019-01-01,WESM,1SNEST_T3L2,139.2,0.0
196098,2019-01-01,WESM,1SNEST_T3L1,275.5,0.0
196097,2019-01-01,WESM,1MSNLO_BATC,0.0,NaN
...,...,...,...,...,...
1889990,2020-07-15,WESM,1MRVEL_T1L1,0.0,0.0
1889891,2020-07-15,WESM,3BAT_BTLEC2,763.7,2.0
1889872,2020-07-15,WESM,3CALIRY_SS,0.0,NaN
1889814,2020-07-15,WESM,1SUAL_T3L1,108.4,4.0


In [74]:
cluster_df[cluster_df['cluster'] == 0].shape
cluster_df[cluster_df['cluster'] == 1].shape
cluster_df[cluster_df['cluster'] == 2].shape
cluster_df[cluster_df['cluster'] == 3].shape
cluster_df[cluster_df['cluster'] == 4].shape
cluster_df[cluster_df['cluster'] == 5].shape

(59, 2)

(17, 2)

(59, 2)

(6, 2)

(3, 2)

(9, 2)

In [76]:
59+17+59+6+3+9

153

In [56]:
Daily_DF_Clustered = Daily_DF.copy()

In [59]:
Daily_DF_Clustered = Daily_DF_Clustered.dropna()
Daily_DF_Clustered

,date,participant,resource,total mw,cluster
0,2019-01-01,WESM,1BALNEO_L01,1089.8,2.0
196100,2019-01-01,WESM,1SUAL_T3L1,109.5,4.0
196099,2019-01-01,WESM,1SNEST_T3L2,139.2,0.0
196098,2019-01-01,WESM,1SNEST_T3L1,275.5,0.0
196096,2019-01-01,WESM,1MEX_CISP,438.4,5.0
...,...,...,...,...,...
1696096,2020-07-15,WESM,1BTPP_T3L2,567.8,0.0
1889990,2020-07-15,WESM,1MRVEL_T1L1,0.0,0.0
1889891,2020-07-15,WESM,3BAT_BTLEC2,763.7,2.0
1889814,2020-07-15,WESM,1SUAL_T3L1,108.4,4.0
